In [1]:
from openai import OpenAI
import os
import json
import time
import Combat

In [2]:
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

In [3]:
f = open('system_prompt.txt','r')
system_prompt = f.read()
f.close()
f = open('world_view', 'r')
world_view = f.read()
f.close()

In [12]:
# # create new assistant object
# assistant = client.beta.assistants.create(
#     name="Story Generator",
#     instructions = system_prompt,
#     response_format={"type": "json_object"},
#     model="gpt-4o",
# )
# ASSISTANT_ID = assistant.id

# Using pre-made assistant
STORY_ID = 'asst_QJULNvgbgB8cU1YgY5JzLAyr'
SKILL_ID = 'asst_JSej8B49OeeI7IwOUNp3dpKY'

In [5]:
player_status = '''{
  'Player Name': 'Argon', 
  'status': {'HP':200, 'MP':100, 
             'Strength':30, 'Skill':15, 
             'Dexterity': 5, 'Intelligence': 5, 
             'Luck': 5, 'Defense': 30, 
             'Speed': 5, 'Concentration': 20, 
             'Reaction': 15, 'MP Regeneration': 2, 'HP Regeneration': 2},
  'Gold': 1000, 
  'Level': 10, 
  'Gender': 'Male', 
  'Social Status': 'Commoner', 
  'Strength Level': '2-star Knight', 
  'Equipment': { 'Top': {'Item Name': 'Ordinary Leather Armor', 'Use Restriction': None, 'Effect': {'Defense':10, 'Dexterity': -3}}, 
                'Weapon': {'Item Name': 'Iron Sword', 'Use Restriction': {'Strength':15}, 'Effect': {'Strength':10}} } 
}'''

In [6]:
start_message = [
  {
    "type": "text",
    "text": world_view
  },
  {
    "type": "text",
    "text": player_status
  }
]

In [21]:
def show_message(json_object):
  if 'Story' in json_object:
    display(json_object['Story'])
    for idx, choice in enumerate(json_object['Choices']):
      print(f'{idx+1}: {choice["text"]} ({choice["status"]}), Gold:({choice["gold"]}), ({choice["next_type"]})')
  else:
    display(json_object)
    
def switching(json_object):
  if 'Combat' in json_object:
    Combat.combat(json_object)
    

def run_thread(user_message, thread, assistant_id):
  run = submit_message(user_message, thread, assistant_id)
  run = wait_run(run, thread)
  messages = get_message(thread)
  text = messages.data[0].content[0].text.value
  json_object=json.loads(text)

  show_message(json_object)
  time.sleep(0.5)
  return json_object


def start_thread(thread, start_message, assistant_id):
  # create thread and add first message in thread
  run_thread(start_message, thread, assistant_id)

  for i in range(10):
    user_message = input()
    print(user_message)
    if user_message == "quit":
      break
    response = run_thread(user_message, thread, assistant_id)
    if 'Combat' in response:
      run_thread(str(response['Combat']), thread, SKILL_ID)


def submit_message(user_message, thread, assistant_id):
  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
  )
  run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )
  return run


def wait_run(run, thread):
  while run.status == "queued" or run.status == "in_progress":
    # connect thread and assistant
    run = client.beta.threads.runs.retrieve(
      thread_id=thread.id,
      run_id=run.id
    )
    time.sleep(0.5)
  return run

def get_message(thread):
  return client.beta.threads.messages.list(thread_id=thread.id)

In [22]:
thread = client.beta.threads.create()

In [23]:
start_thread(thread,start_message,STORY_ID)

"Argon found himself standing at the edge of the ominous Blackwood Forest, a place known for its malevolent spirits and rogue demons. As a 2-star knight, he had faced many challenges, but the tales of creatures lurking within the forest sent shivers down his spine. It was said that a particularly powerful **Rogue Demon** was preying on travelers and disrupting the local ecosystem. Upon entering, the forest felt alive, whispering secrets as the wind howled through the twisted trees. Every step Argon took seemed to echo, amplifying the foreboding atmosphere. After some time navigating the dense woods, he came across an ancient, crumbling statue—a sentinel from a forgotten age. Pausing to catch his breath, Argon's ears picked up a rustling noise from the shadows. Something, or someone, was watching him."

1: Investigate the rustling noise ({'HP': -10, 'Reaction': 5}), Gold:(0), (Combat)
2: Ignore it and proceed deeper into the forest ({'Concentration': -5}), Gold:(0), (Story)
1


{'Combat': {'Monster Name': 'Rogue Demon',
  'Monster Description': 'A malevolent creature with burning red eyes and charred wings, cloaked in dark mist. It preys on travelers, disrupting the natural balance of Blackwood Forest.',
  'status': {'HP': 300,
   'MP': 50,
   'Strength': 40,
   'Skill': 25,
   'Dexterity': 10,
   'Intelligence': 20,
   'Luck': 5,
   'Defense': 20,
   'Speed': 10,
   'Concentration': 15,
   'Reaction': 10,
   'MP Regeneration': 1,
   'HP Regeneration': 1}},
 'Reward': {'gold': 200, 'exp': 150},
 'Item': {'Item types': 'Weapon',
  'Item Name': 'Demon Slasher',
  'Item Description': 'A cursed sword forged in the Underworld. It is exceptionally efficient against demonic beings.',
  'Use Restriction': {'Strength': 25},
  'Effect': {'Strength': 25, 'Luck': -5}}}

{'Attack_Type': 'Magical',
 'Magical_Attack_Method': 'Explosion',
 'Penetration': {'Physical_Defense_Penetration': '15%',
  'Magical_Defense_Penetration': '10%'},
 'Accuracy': '80%',
 'Status_Effect': {'Effect_Type': ['Burn'],
  'Effect_Details': {'Bleed_Poison_Burn_Freeze': {'Status': ['Intelligence'],
    'Coefficient': ['0.2'],
    'Duration': '3 turns'}}},
 'Damage': {'Status': ['Intelligence', 'Skill'],
  'Coefficient': ['2.0', '1.5'],
  'Count': '1'}}

victory


...

"As Argon began to climb the creaking spiral staircase, each step seemed to echo louder than the last. The eerie light above grew brighter as he ascended, casting long, distorted shadows on the walls. When he reached the top, he found himself in a large chamber filled with strange apparatus and bubbling potions. In the center of the room, floating above a stone pedestal, was an orb radiating a mesmerizing light. Argon could feel the immense magical energy emanating from it. As he stepped closer, a spectral figure materialized, blocking his path. It was the spectral guardian of the tower and former apprentice of the mage who once resided here, intent on protecting its master's secrets."

1: Engage the spectral guardian in combat ({'Reaction': 10}), Gold:(0), (Combat)
2: Try to communicate with the spectral guardian ({'Intelligence': 10}), Gold:(0), (Story)
3: Attempt to bypass the spectral guardian and grab the orb ({'Dexterity': 10, 'Luck': 5}), Gold:(0), (Story)
1


{'Combat': {'Monster Name': 'Spectral Guardian',
  'Monster Description': "An ethereal figure dressed in the tattered remnants of a mage's robe. The guardian wields a spectral staff and glows with an eerie, otherworldly light.",
  'status': {'HP': 250,
   'MP': 100,
   'Strength': 20,
   'Skill': 30,
   'Dexterity': 15,
   'Intelligence': 35,
   'Luck': 10,
   'Defense': 15,
   'Speed': 20,
   'Concentration': 25,
   'Reaction': 20,
   'MP Regeneration': 5,
   'HP Regeneration': 2}},
 'Reward': {'gold': 300, 'exp': 200},
 'Item': {'Item types': 'Ring',
  'Item Name': 'Ring of Arcane Mastery',
  'Item Description': 'A ring imbued with the wisdom and power of the ancient mage. Enhances magical capabilities and intelligence.',
  'Use Restriction': {'Intelligence': 20},
  'Effect': {'Intelligence': 10, 'MP Regeneration': 3}}}

{'Attack_Type': 'Magical',
 'Magical_Attack_Method': 'Blast',
 'Penetration': {'Physical_Defense_Penetration': '5%',
  'Magical_Defense_Penetration': '25%'},
 'Accuracy': '90%',
 'Status_Effect': {'Effect_Type': ['Silence'],
  'Effect_Details': {'Stun_Paralysis_Sleep_Silence_Confusion_Blindness_Charm_Fear_Root': {'Duration': '2 turns'}}},
 'Damage': {'Status': ['Intelligence', 'Skill'],
  'Coefficient': ['2.5', '1.8'],
  'Count': '1'}}

quit
